In [1]:
#ライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt

## フーリエトランスフォームサロゲート

非線形統計量として並進誤差を利用

In [2]:
#x:時系列，t:時間遅れ，m:埋め込み次元，nn:使用する状態空間上の近傍点数，stp:並進誤差算出次のステップ数，data:作成するサロゲートデータ数
#Q:並進誤差算出次のサンプリングする状態空間上の点数，A:並進誤差算出処理の回数

def surrogate(x,t,m,nn,stp,data,Q,A=10):
  N = len(x)

  #サロゲートデータの非線形統計量保存用の配列
  Es = np.zeros(data)


  x = np.array(x)

  #時系列データのデータ点数の偶奇で分ける
  if N%2 == 0:
    n = int(N/2)
    Judge = 0
  else:
    n = int((N-1)/2 + 1)
    Judge = 1

  for d in range(data+1):
    xs = np.copy(x)

    #d=0は元データ，それ以外はサロゲートデータ
    if d != 0:

      #フーリエ変換
      xs = np.fft.fft(xs)
      #位相をランダマイズ
      xs = xs[:n] * np.exp(np.random.rand(n)*2*np.pi*1j)

      if Judge == 0:
        xs = np.append(xs,xs[::-1])
      else:
        xs = np.append(xs,xs[-2::-1])

      #逆フーリエ変換
      xs = np.real(np.fft.ifft(xs))

    #埋め込み
    X = np.zeros((N-(m-1)*t,m))
    for i in range(1,m+1):
      X[:,i-1] = xs[(i-1)*t:i*t+N-m*t]


    #並進誤差A回分を格納する配列
    E = np.zeros(A)

    #A回処理開始
    for a in range(A):

      #サンプリング数ごとに算出される値を格納する配列
      e = np.zeros(Q)
      #eに格納するためのインデックス
      j = 0

      sample = np.random.choice(len(X)-stp,Q,replace=False)
      for i in sample:

        #データ点数を超えないように処理
        dist = np.zeros(len(X)-stp)

        #対象のデータ点と状態空間上の任意の点の距離を計算
        dist[:] = np.linalg.norm(X[i,:]-X[:len(X)-stp,:],axis=1)

        #距離の短い順にindexを取得
        di = np.argsort(dist)

        #推移ベクトル用
        v = np.zeros(nn+1)

        #対象のデータ点と近傍点nn個のstp数先の点との推移ベクトルを算出
        v = X[di[:nn+1]+stp,:]-X[di[:nn+1],:]

        #求めた推移ベクトルの平均ベクトルを算出
        v_ave = np.mean(v,axis=0)

        #それぞれデータ点と求めた平均との差分を算出
        v_var = np.sum(np.linalg.norm(v[:]-v_ave,ord=2,axis=1))

        #vをv_aveとv_varを使って算出されるスカラーに更新
        v = v_var/np.linalg.norm(v_ave,ord=2)

        #サンプリング１回で最終的に算出される値を格納
        e[j] = v/(nn+1)
        j += 1

      #サンプリング回数ごとに算出された最終値の中央値を格納
      E[a] = np.median(e)


    #オリジナルデータの非線形統計量
    if d == 0:
      Eo = np.mean(E)
    #サロゲートデータの非線形統計量を配列で保存
    else:
      Es[d-1] = np.mean(E)

    print(str(d)+'回目終了')

  return Eo,Es


In [ ]:
#Lorenz
from mpl_toolkits.mplot3d import Axes3D
p = 10
r = 28
b = 8/3

def f1(x,y):
  return -p*x+p*y
def f2(x,y,z):
  return -x*z+r*x-y
def f3(x,y,z):
  return x*y-b*z

dt = 0.01
x = 0.2
y = 0.3
z = 0.4
X = []
Y = []
Z = []

for i in range(5000):
  X.append(x)
  Y.append(y)
  Z.append(z)
  k1x = f1(x,y)*dt
  k1y = f2(x,y,z)*dt
  k1z = f3(x,y,z)*dt

  k2x = f1(x+0.5*k1x,y+0.5*k1y)*dt
  k2y = f2(x+0.5*k1x,y+0.5*k1y,z+0.5*k1z)*dt
  k2z = f3(x+0.5*k1x,y+0.5*k1y,z+0.5*k1z)*dt

  k3x = f1(x+0.5*k2x,y+0.5*k2y)*dt
  k3y = f2(x+0.5*k2x,y+0.5*k2y,z+0.5*k2z)*dt
  k3z = f3(x+0.5*k2x,y+0.5*k2y,z+0.5*k2z)*dt

  k4x = f1(x+k3x,y+k3y)*dt
  k4y = f2(x+k3x,y+k3y,z+k3z)*dt
  k4z = f3(x+k3x,y+k3y,z+k3z)*dt

  x += (k1x+2*k2x+2*k3x+k4x)/6
  y += (k1y+2*k2y+2*k3y+k4y)/6
  z += (k1z+2*k2z+2*k3z+k4z)/6

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#ax = Axes3D(fig)

ax.set_xlabel("x",fontsize=14)
ax.set_ylabel("y",fontsize=14)
ax.set_zlabel("z",fontsize=14)
ax.plot(X,Y,Z)
plt.show()


In [ ]:
#Lorenz方程式のYを使用

eo,es = surrogate(Y,7,3,5,12,100,100,100)
histogram = plt.hist(es,bins=100,ec='black',label="surrogate")

limit = np.max(histogram[0])+1

plt.bar(eo,height=np.max(histogram[0])+1,width=0.003,color='orange',label="original")

plt.legend()
plt.ylim(0,limit)